In [22]:
from keras.datasets import mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [23]:
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)

scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [25]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),strides=(1,1), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=SGD(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

modelpath = "./model_optimization/mnist_CNN.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [28]:
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_split=0.2, callbacks=[early_stopping_callback, checkpointer])

Epoch 1/100
1499/1500 [============================>.] - ETA: 0s - loss: 1.6066 - accuracy: 0.5802
Epoch 1: val_loss improved from inf to 0.46278, saving model to ./model_optimization\mnist_CNN.hdf5
1500/1500 [==============================] - 42s 28ms/step - loss: 1.6058 - accuracy: 0.5804 - val_loss: 0.4628 - val_accuracy: 0.8657
Epoch 2/100
1500/1500 [==============================] - ETA: 0s - loss: 0.3755 - accuracy: 0.8895
Epoch 2: val_loss improved from 0.46278 to 0.28256, saving model to ./model_optimization\mnist_CNN.hdf5
1500/1500 [==============================] - 46s 31ms/step - loss: 0.3755 - accuracy: 0.8895 - val_loss: 0.2826 - val_accuracy: 0.9171
Epoch 3/100
1499/1500 [============================>.] - ETA: 0s - loss: 0.2849 - accuracy: 0.9154
Epoch 3: val_loss improved from 0.28256 to 0.23715, saving model to ./model_optimization\mnist_CNN.hdf5
1500/1500 [==============================] - 47s 31ms/step - loss: 0.2848 - accuracy: 0.9155 - val_loss: 0.2372 - val_accurac

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test, y_test)
print(f"accuracy : {round(score[1],2)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_val_loss = history.history["val_loss"]
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_val_loss, marker='.', c='red', label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='Trainset_loss')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()